In [1]:
%%capture
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline

In [2]:
%%capture
import sys
sys.path.append('/home/wrwt/Programming/pygraphmodels')
import graphmodels as gm

In [3]:
import warnings
warnings.filterwarnings('ignore', module='matplotlib')
warnings.filterwarnings('ignore', module='IPython')

In [4]:
def bic_score(x, pa):
    l = gm.information.discrete_mutual_information(data[[x]], data[pa]) - gm.information.discrete_entropy(data[[x]])
    n = data.shape[0]
    k = np.prod(true_dgm.factor(x).table.shape) - 1
    result = -n*l + 0.5 * np.log(n) * k
    return result

In [5]:
from os import listdir
import os.path
NETWORKS_PATH = '/home/wrwt/Programming/pygraphmodels/networks/'
network_filenames = listdir(NETWORKS_PATH)
true_dgm = gm.DGM.read(os.path.join(NETWORKS_PATH, 'earthquake.bif'))
true_dgm.draw()

/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):


In [154]:
data = true_dgm.rvs(100000)

In [155]:
from graphmodels.information import discrete_entropy, discrete_mutual_information, factor_mutual_information

In [156]:
discrete_mutual_information(data[['Alarm']], data[['Burglary']])

0.038894484259644563

In [157]:
discrete_mutual_information(data[['Alarm']], data[['Burglary', 'Earthquake']])

0.05966472681371339

In [158]:
discrete_mutual_information(data[['Alarm']], data[['Earthquake']])

0.0136464374789157

In [159]:
discrete_mutual_information(data[['Burglary']], data[['Earthquake']])

8.4000514533765538e-06

In [160]:
def bic_score(dgm, data):
    result = 0.
    for x in dgm.nodes():
        pa = dgm.predecessors(x)
        
        def n_values(x):
            return len(data[x].value_counts())
    
        k = n_values(x)*np.prod([n_values(pa_i) for pa_i in pa]) - 1
        n = data.shape[0]
        l = n*(discrete_mutual_information(data[[x]], data[pa]) - \
               discrete_entropy(data[[x]]))
        result += l - 0.5 * np.log(n) * k
    return result

In [170]:
from itertools import permutations
dgm = gm.DGM(true_dgm)

scores = []
ops = []
for src, dst in permutations(dgm.nodes(), 2):
    if dgm.has_edge(src, dst):
        dgm.remove_edge(src, dst)
        if nx.is_directed_acyclic_graph(dgm):
            scores.append(bic_score(dgm, data))
            ops.append(('remove', src, dst))
        dgm.add_edge(dst, src)
        if nx.is_directed_acyclic_graph(dgm):
            scores.append(bic_score(dgm, data))
            ops.append(('reverse', src, dst))
        dgm.remove_edge(dst, src)
        dgm.add_edge(src, dst)
    else:
        dgm.add_edge(src, dst)
        if nx.is_directed_acyclic_graph(dgm):
            scores.append(bic_score(dgm, data))
            ops.append(('add', src, dst))
        dgm.remove_edge(src, dst)

In [171]:
bic_score(dgm, data)

-44026.543691382198

In [175]:
max(scores)

-44037.216611701828

Как видно, правильная сеть является локальным максимумом BIC score.

In [174]:
ops[np.argmax(scores)]

('add', 'Earthquake', 'Burglary')